In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, min, max, rand, randn

In [3]:
spark = SparkSession \
        .builder \
        .appName('demo') \
        .getOrCreate()

In [4]:
# alt ways to create a spark dataframe
a = spark.createDataFrame(["10","11","13"], "string").toDF("age")
a.show()
#alt_way_b = spark.createDataFrame([("10", ), ("11", ), ("13",  )], ["age"])
#from pyspark.sql.types import StringType
#spark.createDataFrame(["10","11","13"], StringType()).toDF("age")

Py4JJavaError: An error occurred while calling o62.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:575)
	at org.apache.spark.api.python.BasePython

In [6]:
# df = spark.range(0, 10)
# df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()

df = spark \
        .range(0, 10) \
        .withColumn('uniform', rand(seed=13)) \
        .withColumn('normal', randn(seed=31)) \
        .withColumn('normal', randn(seed=31))

In [7]:
#df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()

In [8]:
new_row = spark.createDataFrame([(10, 1, 1)], ['id', 'uniform', 'normal'])
df = df.union(new_row)

In [9]:
df.printSchema()
df.show()

root
 |-- id: long (nullable = true)
 |-- uniform: double (nullable = true)
 |-- normal: double (nullable = true)

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.44292918521277047|-0.15116027592854422|
|  1| 0.8898784253886249| -0.3785563841011868|
|  2| 0.2731073068483362|-0.27674189870783683|
|  3| 0.7784518091224375|  0.6037690932383759|
|  4|0.43776394586845413|  0.9692561472263869|
|  5|   0.87079354700073|-0.18575112254167045|
|  6| 0.8729462507631428|-0.41767215413185804|
|  7|0.27149331793166864|   0.734722467897308|
|  8|0.06415762022697513|-0.30123700668427145|
|  9| 0.9060003277875414|  -0.665029215988346|
| 10|                1.0|                 1.0|
+---+-------------------+--------------------+



In [10]:
df.select([mean('uniform'), min('uniform'), max('uniform')]).show()

+------------------+-------------------+------------+
|      avg(uniform)|       min(uniform)|max(uniform)|
+------------------+-------------------+------------+
|0.6188656123773346|0.06415762022697513|         1.0|
+------------------+-------------------+------------+



In [14]:
# sample covariance of two columns
df.stat.cov('uniform', 'normal')

-0.06948283612954509

In [65]:
# events
e_cols = ["EVENTID","EVENTTIMESTAMP","EVENTTYPE","PROVIDERID","PATIENTID","TASKID","ENCOUNTERID","WORKSTATIONID","PROCESS_ID","EVENTACTION"]
e_rows = [
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","10003000","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","20000073","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","","prd-32835774","QUERY")
]

# workstations
w_cols = ["WORKSTATIONID","WORKSTATIONAREA","EXTERNAL_NAME","DEPARTMENT_ID","NETWORK","COMPANY"]
w_rows = [
    ("10003000","BDW ORTHO/NEURO","Brandywine Hospital Ortho/Neuro","107001009","Brandywine","Brandywine Hospital"),
    ("20000073","FAMILY FLOURTOWN THMG","Springfield Medical Associates","102409001","Chestnut Hill","CHN - Chestnut Hill Clinic"),
    ("00195B2FF60701080B160C03020E1809","EMH EMERGENCY","EMH Emergency Department","10101100","",""),
    ("40001111","PHX 2AR","Rehabilitation at Phoenixville Hospital","107601028","Phoenixville","Phoenixville Hospital"),
    ("50001297","PTN MAIN OR","Pottstown Main OR","107801016","Pottstown","Pottstown Hospital"),
    ("50001303","PTN EMERGENCY","Pottstown Hospital Emergency Department","107801001","Pottstown","Pottstown Hospital"),
    ("50001310","INTL MED BOYER THMG","Berks Medical Associates","103201001","Pottstown","PTS - Pottstown Clinic Co")
]

e_df = spark.createDataFrame(e_rows, e_cols)
w_df = spark.createDataFrame(w_rows, w_cols)

lte_df = w_df.select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [66]:
e_df.printSchema()
e_df.show()

root
 |-- EVENTID: string (nullable = true)
 |-- EVENTTIMESTAMP: string (nullable = true)
 |-- EVENTTYPE: string (nullable = true)
 |-- PROVIDERID: string (nullable = true)
 |-- PATIENTID: string (nullable = true)
 |-- TASKID: string (nullable = true)
 |-- ENCOUNTERID: string (nullable = true)
 |-- WORKSTATIONID: string (nullable = true)
 |-- PROCESS_ID: string (nullable = true)
 |-- EVENTACTION: string (nullable = true)

+-----------------+-------------------+---------+----------+---------+------+-----------+-------------+------------+-----------+
|          EVENTID|     EVENTTIMESTAMP|EVENTTYPE|PROVIDERID|PATIENTID|TASKID|ENCOUNTERID|WORKSTATIONID|  PROCESS_ID|EVENTACTION|
+-----------------+-------------------+---------+----------+---------+------+-----------+-------------+------------+-----------+
|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|     10003000|prd-32835774|      QUERY|
|5636059882.643001|2019-08-07 00:11:22|     View|     1417

In [ ]:
foo = w_df.filter()select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [67]:
newRow = spark.createDataFrame([("4", "ASD")], ['WORKSTATIONID', 'NETWORK'])
appended = lte_df.union(newRow)
appended.show()

# .filter(df.age > 3)

+--------------------+-------------+
|       WORKSTATIONID|      NETWORK|
+--------------------+-------------+
|            10003000|   Brandywine|
|            20000073|Chestnut Hill|
|00195B2FF60701080...|             |
|            40001111| Phoenixville|
|            50001297|    Pottstown|
|            50001303|    Pottstown|
|            50001310|    Pottstown|
|                   4|          ASD|
+--------------------+-------------+



In [ ]:
w_df.printSchema()
w_df.show()

In [68]:
df = e_df.join(lte_df, "WORKSTATIONID", "right_outer")

In [75]:
df.filter(df['EVENTID'] != '').show()

+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+
|WORKSTATIONID|          EVENTID|     EVENTTIMESTAMP|EVENTTYPE|PROVIDERID|PATIENTID|TASKID|ENCOUNTERID|  PROCESS_ID|EVENTACTION|      NETWORK|
+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+
|     20000073|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|prd-32835774|      QUERY|Chestnut Hill|
|     10003000|5636059882.643001|2019-08-07 00:11:22|     View|     14173|  Z854894| 17008|30041094735|prd-32835774|      QUERY|   Brandywine|
+-------------+-----------------+-------------------+---------+----------+---------+------+-----------+------------+-----------+-------------+



In [ ]:
from pyspark.sql.functions import round
a = df.groupBy('NETWORK').count()

def pctVal(x):
    #return round(a['count']/df.count(),2)
    b = round(x/df.count(),2)
    return b
    
a.withColumn('pct', round(a['count']/df.count(), 2)).show()

In [ ]:
def func(v):
    print(a[v])
    return v

a.select('NETWORK', func('count')).show()